# Register the model in the MLFlow server

Use the MlflowClient instance to:

- Register a new version for the experiment green-taxi-trip-duration
- Retrieve the latests versions of the model 
- Transition to "Production" and adding annotations to it.

In [1]:
import mlflow
from mlflow.tracking.client import MlflowClient
import os 
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()
MLFLOW_TRACKING_URI = os.environ.get("MLFLOW_TRACKING_URI")
RUN_ID = os.environ.get("RUN_ID")
name="yellow-taxi-ride-duration"

Connect to the MLFlow server, register the model

In [2]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
model_uri = f"runs:/{RUN_ID}/model"
mlflow.register_model(model_uri=model_uri, name=name)

Registered model 'yellow-taxi-ride-duration' already exists. Creating a new version of this model...
2023/07/19 15:14:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: yellow-taxi-ride-duration, version 5
Created version '5' of model 'yellow-taxi-ride-duration'.


<ModelVersion: aliases=[], creation_timestamp=1689772478049, current_stage='None', description='', last_updated_timestamp=1689772478049, name='yellow-taxi-ride-duration', run_id='728c2a4ec38b4838a5e815becb56ca29', run_link='', source='gs://mlflow-artifacts-cjh/default/2/728c2a4ec38b4838a5e815becb56ca29/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

Use the MlflowClient to get more information about the model:

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
model_name = name
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 5, stage: None
version: 2, stage: Production


We see we only have a version 1 of the model. If we would have registered more artifacts as models under the same name, we would have seen more versions here.
But we want to change the stage from `None` to `Production` and add a description to it. 

First change the stage:

In [5]:
model_version = latest_versions[0].version
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1689772478049, current_stage='Production', description='', last_updated_timestamp=1689772478449, name='yellow-taxi-ride-duration', run_id='728c2a4ec38b4838a5e815becb56ca29', run_link='', source='gs://mlflow-artifacts-cjh/default/2/728c2a4ec38b4838a5e815becb56ca29/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

Add a description:

In [6]:
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1689772478049, current_stage='Production', description='The model version 5 was transitioned to Production on 2023-07-19', last_updated_timestamp=1689772478552, name='yellow-taxi-ride-duration', run_id='728c2a4ec38b4838a5e815becb56ca29', run_link='', source='gs://mlflow-artifacts-cjh/default/2/728c2a4ec38b4838a5e815becb56ca29/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

Check out the model again to see if the stage change and description were added:

In [7]:
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}, description: {version.description}")

version: 4, stage: None, description: 
version: 5, stage: Production, description: The model version 5 was transitioned to Production on 2023-07-19
